In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

sns.set()

In [ ]:
# Пункт 1
# Создадим класс с вызовом функции косинуса
class FirstModel:
  def __init__(self):
    self.a = tf.Variable(3.0)
    self.b = tf.Variable(7.0)

  def __call__(self, x):
    return tf.cos(self.a * x + self.b)

In [ ]:
# Создаем датасет
model = FirstModel()
dset = tf.data.Dataset.from_tensor_slices(tf.random.normal(shape=(1000,)))
share_dat = dset.batch(25)
out_dat = dset.map(model).batch(25)
print(share_dat)

In [ ]:
# Пункт 2
TRUE_W = 4.0
TRUE_b = 6.0
num_examples = 1000

inputs = tf.random.normal(shape=[num_examples])
noise = tf.random.normal(shape=[num_examples])
outputs = tf.cos(inputs * TRUE_W + TRUE_b * noise)

# Функция ошибки
def loss(target_y, pred_y):
  return tf.reduce_mean(tf.square(target_y - pred_y))

plt.figure(figsize=(20, 10))
plt.scatter(inputs, outputs, color='r')
plt.scatter(inputs, model(inputs), color='b')
plt.show()

print(f'\nCurrent loss: {loss(model(inputs), outputs).numpy()}')

In [ ]:
# Пункт 3
# Функция обучения модели
def train(model, inputs, outputs, learning_rate):
  with tf.GradientTape() as t:
    curr_loss = loss(outputs, model(inputs))
    da, db = t.gradient(curr_loss, [model.a, model.b])
    model.a.assign_sub(learning_rate * da)
    model.b.assign_sub(learning_rate * db)


In [ ]:
!rm '/tmp/early/'

In [ ]:
tf.__version__

In [ ]:
PATH = '/tmp/early/'

cur_time = datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
loss_log_dir = PATH + cur_time + '/data'
loss_sum_writter = tf.summary.create_file_writer(loss_log_dir)

model = FirstModel()

with loss_sum_writter.as_default():
  Ws, bs = [], []
  for epoch in range(1000):
    Ws.append(model.a.numpy())
    bs.append(model.b.numpy())
    cur_loss = loss(outputs, model(inputs))
    train(model, inputs, outputs, learning_rate=0.01)
    print(f'Epoch: {epoch: 2d} a={Ws[-1] : 1.2f}, loss={cur_loss}')
    tf.summary.scalar('Current loss', cur_loss, step=epoch)
    tf.summary.scalar('a', Ws[-1], step=epoch)
    tf.summary.scalar('b', bs[-1], step=epoch)
    loss_sum_writter.flush()

plt.figure(figsize=(16, 8))
plt.plot(range(epoch+1), Ws, 'r', range(epoch+1), bs, 'g')
plt.plot([TRUE_W]*epoch, 'r--', [TRUE_b]*epoch, 'b--')
plt.legend(['a', 'b', 'True a', 'True b'])
plt.show()

In [ ]:
%load_ext tensorboard

In [ ]:
!ls /tmp/early/

In [ ]:
%tensorboard --logdir /tmp/early/